## Coin Toss Streaks
Ben Cook, January 2017

In [1]:
import itertools as it

In [4]:
import pandas as pd
import numpy as np

In [2]:
pH = 0.6
pT = 1.0 - pH


### Construct all possible length-10 sequences

In [5]:
asdf = pd.Series(['H','T']).to_frame()
asdf.columns = ['results']
for _ in range(1,10):
    asdf = pd.concat((asdf + 'H', asdf + 'T'))
print asdf.head()
len(asdf)

      results
0  HHHHHHHHHH
1  THHHHHHHHH
0  HTHHHHHHHH
1  TTHHHHHHHH
0  HHTHHHHHHH


1024

In [6]:
asdf['first'] = asdf.results.map(lambda s: s[0])
asdf['last'] = asdf.results.map(lambda s: s[-1])
asdf.head()

,results,first,last
0,HHHHHHHHHH,H,H
1,THHHHHHHHH,T,H
0,HTHHHHHHHH,H,H
1,TTHHHHHHHH,T,H
0,HHTHHHHHHH,H,H


In [7]:
asdf['nHeads'] = asdf.results.map(lambda s: s.count('H'))
asdf['nTails'] = asdf.results.map(lambda s: s.count('T'))
asdf.head()

,results,first,last,nHeads,nTails
0,HHHHHHHHHH,H,H,10,0
1,THHHHHHHHH,T,H,9,1
0,HTHHHHHHHH,H,H,9,1
1,TTHHHHHHHH,T,H,8,2
0,HHTHHHHHHH,H,H,9,1


In [8]:
asdf['prob'] = np.power(0.6,asdf.nHeads) * np.power(0.4,asdf.nTails)
print asdf.prob.sum()
asdf.head()

1.0


,results,first,last,nHeads,nTails,prob
0,HHHHHHHHHH,H,H,10,0,0.006047
1,THHHHHHHHH,T,H,9,1,0.004031
0,HTHHHHHHHH,H,H,9,1,0.004031
1,TTHHHHHHHH,T,H,8,2,0.002687
0,HHTHHHHHHH,H,H,9,1,0.004031


In [9]:
iterLength = lambda iterator: sum((1 for _ in iterator))
asdf['nStreaks'] = asdf.results.map(lambda s: iterLength(it.groupby(s)))
asdf.head()

,results,first,last,nHeads,nTails,prob,nStreaks
0,HHHHHHHHHH,H,H,10,0,0.006047,1
1,THHHHHHHHH,T,H,9,1,0.004031,2
0,HTHHHHHHHH,H,H,9,1,0.004031,3
1,TTHHHHHHHH,T,H,8,2,0.002687,2
0,HHTHHHHHHH,H,H,9,1,0.004031,3


#### Aggregate by number of streaks

In [46]:
byStreaks = asdf.groupby('nStreaks')
for (n,group) in byStreaks:
    print n, group.prob.sum()
prob1 = sum((n*group.prob.sum() for (n,group) in byStreaks))
print "%0.10f" %(prob1)

1 0.0061514752
2 0.0235573248
3 0.0942292992
4 0.1696555008
5 0.2544832512
6 0.2252095488
7 0.1501396992
8 0.0599851008
9 0.0149962752
10 0.0015925248
5.3200000000


#### probability of 6 or more streaks

In [48]:

byStreaks = asdf.groupby(('nStreaks'))['prob'].sum()
prob3 = byStreaks[6:]
print prob3
print "%0.10f" % (prob3.sum())

nStreaks
7     0.150140
8     0.059985
9     0.014996
10    0.001593
Name: prob, dtype: float64
0.2267136000


#### Probability of >6 streaks given >5

In [44]:
# Problem 5
print byStreaks[6:].sum() / byStreaks[5:].sum()

0.501664056382


#### Probability of 6 or more streaks & 6 or more heads

In [45]:
# Problem 7
asdf['prob'][(asdf['nStreaks'] > 5) & (asdf['nHeads'] > 5)].sum()

0.23887872000000043

### Convert to a form to aggregate into longer sequences
(brute-force solution grows exponentially in sequence length)

In [13]:
summary = asdf.drop('results',1).drop('nTails',1).groupby(('first','last','nHeads','nStreaks'), as_index=False).aggregate(np.sum)
print summary.head()
print len(summary)

  first last  nHeads  nStreaks      prob
0     H    H       2         3  0.000236
1     H    H       3         3  0.000708
2     H    H       3         5  0.002123
3     H    H       4         3  0.001593
4     H    H       4         5  0.007963
92


In [14]:
def combine(row1,row2):
    row1 = row1[1]
    row2 = row2[1]
    first = row1['first']
#    first = row1[0]
    last = row2['last']
#    last = row2[0]
    nHeads = row1['nHeads'] + row2['nHeads']
#    nHeads = row1[2] + row2[2]
    nStreaks = row1['nStreaks'] + row2['nStreaks']
#    nStreaks = row1[3] + row2[3]
    if row1['last'] == row2['first']:
#    if row1[1] == row2[0]:
        nStreaks -= 1
    prob = row1.prob * row2.prob
#    prob = row1[4] * row2[4]
    return (first, last, nHeads, nStreaks, prob)

def combineDict(row1,row2):
    ans = dict()
    ans['nHeads'] = row1.nHeads + row2.nHeads
    nStreaks = row1.nStreaks + row2.nStreaks
    if row1['last'] == row2['first']:
        nStreaks -= 1
    ans['nStreaks'] = nStreaks
    ans['first'] = row1['first']
    ans['last'] = row2['last']
    ans['prob'] = row1.prob * row2.prob
    return ans

In [16]:
table20 = pd.DataFrame.from_records((combine(row1,row2)
                                     for (row1,row2) in it.product(summary.iterrows(), summary.iterrows())),
                                   columns= summary.columns)
#table20 = pd.DataFrame.from_records((combine(row1,row2) for row1,row2 in it.product(summary,summary)))
table20 = table20.groupby(('first','last','nHeads','nStreaks'), as_index=False).aggregate(np.sum)

print len(table20)
table20.head()

382


,first,last,nHeads,nStreaks,prob
0,H,H,2,3,2.473901e-08
1,H,H,3,3,7.421703e-08
2,H,H,3,5,5.937363e-07
3,H,H,4,3,1.669883e-07
4,H,H,4,5,2.504825e-06


In [20]:
%%time
table40 = pd.DataFrame.from_records((combine(row1,row2)
                                     for (row1,row2) in it.product(table20.iterrows(), table20.iterrows())),
                                   columns= summary.columns)
table40 = table40.groupby(('first','last','nHeads','nStreaks'), as_index=False).aggregate(np.sum)
table40 = table40[table40['prob'] > 1e-9]

print len(table40)
print table40.head()
print table40.prob.sum()

1251
   first last  nHeads  nStreaks          prob
25     H    H       8        11  2.047278e-09
26     H    H       8        13  3.685100e-09
27     H    H       8        15  2.281252e-09
31     H    H       9         9  1.320824e-09
32     H    H       9        11  7.132451e-09
0.999999961937
Wall time: 12.5 s


In [18]:
def aggregate(table1,table2):
    table = pd.DataFrame.from_records((combine(row1,row2)
                                      for (row1,row2) in it.product(table1.iterrows(), table2.iterrows())),
                                     columns = summary.columns)
    table = table.groupby(('first','last','nHeads','nStreaks'), as_index=False).aggregate(np.sum)
    table = table[table['prob'] > 1e-9]
    return table

In [23]:
%%time
table50 = aggregate(table40,summary)
print len(table50)
print table50.prob.sum()
print table50.head()

1701
0.999999905101
   first last  nHeads  nStreaks          prob
23     H    H      12        15  1.173856e-09
24     H    H      12        17  2.710016e-09
25     H    H      12        19  3.447089e-09
26     H    H      12        21  2.241784e-09
31     H    H      13        15  3.672388e-09
Wall time: 10 s


In [24]:
%%time
table200 = reduce(aggregate, it.repeat(summary,20))
print len(table200)
print table200.prob.sum()
print table200.head()

7770
0.999996656153
   first last  nHeads  nStreaks          prob
65     H    H      84        95  1.301020e-09
66     H    H      84        97  1.538681e-09
67     H    H      84        99  1.591121e-09
68     H    H      84       101  1.473276e-09
69     H    H      84       103  1.191318e-09
Wall time: 9min 29s


In [25]:
summary200 = table200.drop('first',1).drop('last',1).groupby(('nHeads','nStreaks'),as_index=False).aggregate(np.sum)
print len(summary200)
print summary200.prob.sum()
print summary200.head()

3967
0.999996656153
   nHeads  nStreaks          prob
0      83        93  1.033180e-09
1      83        95  1.308270e-09
2      83        97  1.496805e-09
3      83        98  1.060768e-09
4      83        99  1.566137e-09


#### Probability of exceeding 100 heads & 100 streaks in 200 tosses

In [29]:
prob8 = summary200[(summary200['nHeads'] > 100) & (summary200['nStreaks'] > 100)]
print prob8.prob.sum()

0.29295462316


In [33]:
def combineStreaks(row1,row2):
    row1 = row1[1]
    row2 = row2[1]
    first = row1['first']
    last = row2['last']
    nStreaks = row1['nStreaks'] + row2['nStreaks']
    if row1['last'] == row2['first']:
        nStreaks -= 1
    prob = row1.prob * row2.prob
    return (first, last, nStreaks, prob)


def aggregateStreaks(table1,table2):
    table = pd.DataFrame.from_records((combineStreaks(row1,row2)
                                      for (row1,row2) in it.product(table1.iterrows(), table2.iterrows())),
                                     columns = ('first','last','nStreaks','prob'))
    table = table.groupby(('first','last','nStreaks'), as_index=False).aggregate(np.sum)
    table = table[table['prob'] > 1e-9]
    return table

In [31]:
summaryStreaks = summary.drop('nHeads',1).groupby(('first','last','nStreaks'), as_index=False).aggregate(np.sum)
print len(summaryStreaks)
print summaryStreaks.prob.sum()
print summaryStreaks.head()

20
1.0
  first last  nStreaks      prob
0     H    H         1  0.006047
1     H    H         3  0.073503
2     H    H         5  0.175211
3     H    H         7  0.096082
4     H    H         9  0.009157


In [34]:
%%time
table500 = reduce(aggregateStreaks, it.repeat(summaryStreaks,50))
print len(table500)
print table500.prob.sum()
print table500.head()

260
0.999999859704
  first last  nStreaks          prob
2     H    H       175  1.818587e-09
3     H    H       177  6.840524e-09
4     H    H       179  1.950054e-08
5     H    H       181  4.976351e-08
6     H    H       183  1.201855e-07
Wall time: 12.8 s


In [39]:
summary500 = table500.drop('first',1).drop('last',1).groupby('nStreaks',as_index=False).aggregate(np.sum)
print len(summary500)
print summary500.prob.sum()
print summary500.head()

131
0.999999859704
   nStreaks          prob
0       175  1.818587e-09
1       176  3.809733e-09
2       177  8.815147e-09
3       178  1.334220e-08
4       179  2.598568e-08


#### Expected number of streaks in 500-length sequence

In [49]:
# Problem 2
print "%0.10f" % ((summary500.nStreaks * summary500.prob).sum())

240.5199658425


#### Probability of exceeding 250 streaks

In [50]:
# Problem 4
exceedance =  summary500[summary500['nStreaks'] > 250].sum()
print exceedance
print "%0.10f" % ( exceedance.prob)

nStreaks    15290.000000
prob            0.194905
dtype: float64
0.1949047296


#### Probability of exceeding 250 streaks given >240

In [51]:
# Problem 6
exceedance = summary500[summary500['nStreaks'] > 250].sum() / summary500[summary500['nStreaks'] > 240].sum()
print exceedance
print "%0.10f" % ( exceedance.prob)

nStreaks    0.861651
prob        0.388488
dtype: float64
0.3884878991
